# Generate calibration parameters wflow_sbm
## Modify staticmaps.nc

In [ ]:
from glob import glob
from pathlib import Path

import xarray as xr
from dask.diagnostics import ProgressBar

## Set Paths

In [ ]:
# Snellius paths
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/')
MODELS = Path(f'{ROOT}/wflow/data/')

## Set config

In [ ]:
# Get available basin IDs wflow_sbm
basin_dirs = glob(f'{MODELS}/*')
basin_ids = [s.split('/')[-1] for s in basin_dirs]
basin_ids.sort()

# Set calibration interval
calibration_values = [1,5,10,15,20,
                      25,30,35,40,45,
                      50,55,60,65,70,
                      75,80,85,90,95,
                      100,125,150,175,
                      200,225,250,275,
                      300,350,400,450,550,
                      600,650,700,750,800,
                      850,900,950,1000,1500,
                      2000,2500,3000,4000,
                      4500,5000,7500,10000]

# Generate calibration parameters wflow_sbm

In [ ]:
def modify_staticmaps(modeldir, basin_id, calibration_values):

    # Modify staticmaps.nc
    for calibration_value in calibration_values: 
        # Load dataset
        ds = xr.open_dataset(f'{modeldir}/{basin_id}/staticmaps.nc')
        # Multiply with calibration value
        ds['KsatHorFrac'] = ds.KsatHorFrac * (calibration_value / 100)
        
        # Save to netcdf
        fname = f'{modeldir}/{basin_id}/staticmaps_ksathorfrac_{calibration_value}.nc'
        
        write_job = ds.to_netcdf(fname, compute=False)
        # with ProgressBar():
        write_job.compute()
        
    return print(f'Finished {basin_id}')

In [ ]:
# Run function
for basin_id in basin_ids:
    modify_staticmaps(MODELS, basin_id, calibration_values)